# 掛載 Google Drive

In [ ]:
!pip install google.colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 觀看系統設定

In [ ]:
!lsb_release -a

In [ ]:
!nvidia-smi

In [ ]:
!nvcc -V

In [ ]:
# 切換目錄 (Colab 預設目錄為 /content，使用 %cd 切換目錄)
%cd /content/drive/My Drive/colab_test/

# 微調模型

In [ ]:
# 安裝套件
!pip install torch torchvision torchaudio transformers datasets evaluate accelerate scikit-learn scikit-learn 

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))

In [ ]:
'''
AutoTokenizer：這有助於將我們的文字資料標記為 BERT 可以理解的格式。 「Auto」前綴意味著它可以為各種模型推斷適當的分詞器。

DataCollat​​eWithPadding：確保我們分詞化後的資料，以一致的長度串接在一起，並在必要時增加 padding。這對於訓練的穩定性和效率至關重要。

AutoModelForSequenceClassification：一個通用的類別，是用於「序列分類」任務的模型架構。「Auto」前綴使其在各種預訓練模型中具有通用性。

TrainingArguments：定義訓練配置的設定，例如 learning rateb、batch size 和 epoch。

Trainer：用於訓練和評估，使 finetune 變得簡單。

pipeline：使用模型的模型。
'''
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

import random
from sklearn.metrics import f1_score


'''
函式
'''
# 讀取 .txt 文件
def load_dataset_from_file(file_path, seed=42):
    # 讀檔
    with open(file_path, "r", encoding='utf-8') as file:
        # 將每一行資料以 list 型態回傳
        lines = file.readlines()

        # 洗牌 (記得設定 random seed，確保每次洗牌結果一樣)
        random.seed(seed)
        random.shuffle(lines)

        # 整合訓練資料
        sentences = []
        labels = []
        for line in lines:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                sentences.append(parts[0])
                labels.append(int(parts[1]))
            else:
                print(f'格式錯誤的行號: {line}')
        return sentences, labels
    
# 轉換成 huggingface trainer 可以使用的 datasets
def convert_to_dataset(sentences, labels, tokenizer, max_length):
    # 建立 Dataset
    dataset = Dataset.from_dict({
        'sentences': sentences,
        'labels': labels
    })

    # 回傳切分資料 (訓練 和 驗證)
    dataset = dataset.train_test_split(test_size=0.2)
    '''
    DatasetDict({
        train: Dataset({
            features: ['sentences', 'labels'],
            num_rows: 6212
        })
        test: Dataset({
            features: ['sentences', 'labels'],
            num_rows: 1554
        })
    })
    '''

    # 預處理資料
    def preprocess_data(dataset):
        # 將句子轉換為 token (tokenization)
        return tokenizer(dataset['sentences'], truncation=True, padding=True, return_tensors='pt', max_length=max_length)

    # 轉換資料
    train_data = dataset['train'].map(preprocess_data, batched=True)
    valid_data = dataset['test'].map(preprocess_data, batched=True)

    return DatasetDict({
        'train': train_data,
        'test': valid_data
    })

# 計算模型評估指標
def compute_metrics(predicted_results):
    labels = predicted_results.label_ids
    preds = predicted_results.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='micro') # binary, micro, macro, weighted
    return {
        'f1': f1,
    }

In [ ]:
# 主程式 - 微調模型
if __name__ == "__main__":
    '''
    設定 hyperparameters
    '''
    batch_size = 8 # 批次大小
    learning_rate = 0.00005 # 學習率 5e-5
    epochs = 3 # 訓練次數
    model_name = 'bert-base-chinese' # 預訓練模型名稱
    output_dir = 'output' # 輸出資料夾
    seed = 42 # 隨機種子
    max_seq_length = 512 # 最大長度
    num_labels = 2 # 二元分類

    # 讀取訓練資料
    sentences, labels = load_dataset_from_file('./reviews.txt')

    # 載入 tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 將資料轉換為 huggingface 可以使用的格式
    dataset = convert_to_dataset(sentences, labels, tokenizer, max_length=max_seq_length)

    # 讀取模型
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    # 設定訓練參數
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=2,
        learning_rate=learning_rate,
        warmup_steps=50,
        weight_decay=0.01,
        eval_strategy="steps", # epoch, steps, no
        eval_steps=50,
        save_strategy="steps", # epoch, steps, no
        save_steps=50,
        save_total_limit=3,
        load_best_model_at_end=True,
        seed=seed,
        # lr_scheduler_type="linear", # https://blog.csdn.net/muyao987/article/details/139319466
        # report_to='wandb', # https://wandb.ai/
    )

    # 設定 Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        data_collator=None, # DataCollatorWithPadding(tokenizer),
        compute_metrics=compute_metrics,
    )

    # 開始訓練
    trainer.train()

    # 儲存模型
    trainer.save_model(output_dir)

    # 儲存 tokenizer
    tokenizer.save_pretrained(output_dir)

# 拿微調好的模型，進行預測

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)
from pprint import pprint

model_dir = './output'
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
pipe = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device=0)

list_text = [
    '這個房間真的不錯，服務人員也很親切，下次還會再來！',
    '這個房間真的很爛，服務人員也很差，下次不會再來！',
    '一般般',
]
result = pipe(list_text)
pprint(result)